In [1]:
# Check GPU type
!nvidia-smi

Tue Mar  2 07:08:47 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.39       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   34C    P0    24W / 300W |      0MiB / 16160MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
# Upgrade pip and install ktrain
!pip -qq install -U pip
!pip -qq install ktrain

     |████████████████████████████████| 1.5MB 5.4MB/s 
     |████████████████████████████████| 25.3 MB 94.2 MB/s 
     |████████████████████████████████| 6.8 MB 61.9 MB/s 
     |████████████████████████████████| 981 kB 56.6 MB/s 
     |████████████████████████████████| 263 kB 58.2 MB/s 
     |████████████████████████████████| 1.3 MB 58.2 MB/s 
     |████████████████████████████████| 1.2 MB 60.2 MB/s 
     |████████████████████████████████| 468 kB 27.6 MB/s 
     |████████████████████████████████| 1.1 MB 60.4 MB/s 
     |████████████████████████████████| 883 kB 60.1 MB/s 
     |████████████████████████████████| 2.9 MB 65.3 MB/s 


In [3]:
!gdown --id 1LZBMbdMAr8iwmNfN2JkiFw-uGPleBOsf
!unzip -q '/content/Arabizi_data.zip'

Downloading...
From: https://drive.google.com/uc?id=1LZBMbdMAr8iwmNfN2JkiFw-uGPleBOsf
To: /content/Arabizi_data.zip
3.67MB [00:00, 106MB/s]


In [95]:
# Import libaries
import numpy as np 
import pandas as pd
from tqdm import tqdm
import random
import os
import re
import ktrain
from ktrain import text
import tensorflow as tf
from sklearn.model_selection import StratifiedKFold
import string
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
nltk.download('punkt')
import warnings
warnings.filterwarnings('ignore')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [5]:
# Set seed
SEED = 3031

# def set_seeds(seed=SEED):
#     os.environ['PYTHONHASHSEED'] = str(seed)
#     random.seed(seed)
#     tf.random.set_seed(seed)
#     np.random.seed(seed)

# def set_global_determinism(seed=SEED):
#     set_seeds(seed=seed)

#     os.environ['TF_DETERMINISTIC_OPS'] = '1'
#     os.environ['TF_CUDNN_DETERMINISTIC'] = '1'
    
#     tf.config.threading.set_inter_op_parallelism_threads(1)
#     tf.config.threading.set_intra_op_parallelism_threads(1)

# set_global_determinism(seed=SEED)

In [77]:
def clean_text(text):
    '''Make text lowercase, remove text in square brackets,remove links,remove punctuation
    and remove words containing numbers.'''
    text = text.lower()
    text = re.sub('\[.*?\]', '', text)
    text = re.sub('https?://\S+|www\.\S+', '', text)
    text = re.sub('<.*?>+', '', text)
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    text = re.sub('\n', '', text)
    text = re.sub('\w*\d\w*', '', text)
    return text

def text_preprocessing(text):
    """
    Cleaning and parsing the text.

    """
    tokenizer = nltk.tokenize.RegexpTokenizer(r'\w+')
    nopunc = clean_text(text)
    tokenized_text = tokenizer.tokenize(nopunc)
    #remove_stopwords = [w for w in tokenized_text if w not in stopwords.words('english')]
    combined_text = ' '.join(tokenized_text)
    return combined_text

def text_cleaner(text):
  text = re.sub('\s+',' ', text)
  text = text.strip()
  text = re.sub(r'(.)\1+', r'\1\1', text)
  return text

In [122]:
train = pd.read_csv('/content/Arabizi_data/Train.csv')
test = pd.read_csv('/content/Arabizi_data/Test.csv')
sample = pd.read_csv('/content/Arabizi_data/SampleSubmission.csv')
train.head()

,ID,text,label
0,13P0QT0,3sbaaaaaaaaaaaaaaaaaaaa lek ou le seim riahi o...,-1
1,SKCLXCJ,cha3eb fey9elkoum menghir ta7ayoul ou kressi,-1
2,V1TVXIJ,bereau degage nathef ya slim walahi ya7chiw fi...,-1
3,U0TTYY8,ak slouma,1
4,68DX797,entom titmanou lina a7na 3iid moubarik a7na ch...,-1


In [123]:
train.label.value_counts()

 1    38239
-1    29295
 0     2466
Name: label, dtype: int64

In [124]:
train.label = train.label.astype(str)

In [125]:
# Preview last five rows in test
test.tail()

,ID,text
29995,NHXTL3R,me ihebekch raw
29996,U1YWB2O,nchallah rabi m3ak w iwaf9ek mais just 7abit n...
29997,O3KYLM0,slim rabi m3ak w e5edem w 5alli l7ossed lemnay...
29998,W4C38TY,bara 5alis rouhik yizi mitbal3it jam3iya hlaki...
29999,4NNX5QE,rabi m3aaaak ya khawlaaa n7ebouuuuk rana barsh...


In [126]:
train.shape, test.shape, sample.shape

((70000, 3), (30000, 2), (30000, 2))

In [127]:
tqdm.pandas()
train['clean_text'] = train.text.apply(lambda x: text_cleaner(x))
train.head()

,ID,text,label,clean_text
0,13P0QT0,3sbaaaaaaaaaaaaaaaaaaaa lek ou le seim riahi o...,-1,3sbaa lek ou le seim riahi ou 3sbaa le ca
1,SKCLXCJ,cha3eb fey9elkoum menghir ta7ayoul ou kressi,-1,cha3eb fey9elkoum menghir ta7ayoul ou kressi
2,V1TVXIJ,bereau degage nathef ya slim walahi ya7chiw fi...,-1,bereau degage nathef ya slim walahi ya7chiw fi...
3,U0TTYY8,ak slouma,1,ak slouma
4,68DX797,entom titmanou lina a7na 3iid moubarik a7na ch...,-1,entom titmanou lina a7na 3iid moubarik a7na ch...


In [128]:
test['clean_text'] = test.text.apply(lambda x: text_cleaner(x))
test.head()

,ID,text,clean_text
0,2DDHQW9,barcha aaindou fiha hak w barcha teflim kadhalik,barcha aaindou fiha hak w barcha teflim kadhalik
1,5HY6UEY,ye gernabou ye 9a7ba,ye gernabou ye 9a7ba
2,ATNVUJX,saber w barra rabbi m3ak 5ouya,saber w barra rabbi m3ak 5ouya
3,Q9XYVOQ,cha3ébbb ta7aaaaannnnnnnnnnn tfouuhh,cha3ébb ta7aann tfouuhh
4,TOAHLRH,rabi y5alihoulek w yfar7ek bih w inchallah itc...,rabi y5alihoulek w yfar7ek bih w inchallah itc...


In [129]:
MODEL_NAME = 'bert-base-uncased'
MAX_LEN = 64
BATCH_SIZE = 64
FOLDS = 5
LR = 3e-5
EPOCHS = 3

# List of class names
CLASS_NAMES = sorted(train.label.unique().tolist()) # ['afya', 'burudani', 'kimataifa', 'kitaifa', 'michezo', 'uchumi']

# Instantiate transformer with the provided parameters
t = text.Transformer(model_name=MODEL_NAME, maxlen=MAX_LEN, class_names=CLASS_NAMES, batch_size=BATCH_SIZE)
CLASS_NAMES

['-1', '0', '1']

In [130]:
%%time
# Prepare test data
test_data = np.asarray(test.clean_text)

# Set number of folds to 3
folds = StratifiedKFold(n_splits=FOLDS, random_state=SEED, shuffle=False)

# List to store predictions and loss-score per fold
oof_preds = []
oof_loss_score = []

for i, (train_index, test_index) in enumerate(folds.split(train.clean_text, train.label)):
  X_train, X_test = list(train.loc[train_index, 'clean_text']), list(train.loc[test_index, 'clean_text'])
  y_train, y_test = np.asarray(train.loc[train_index, 'label']), np.asarray(train.loc[test_index, 'label'])

  # Preprocess training and validation data
  train_set = t.preprocess_train(X_train, y_train, verbose = 0)
  val_set = t.preprocess_test(X_test, y_test, verbose = 0)

  # Instantiate model
  model = t.get_classifier()
  learner = ktrain.get_learner(model, train_data=train_set, val_data=val_set, batch_size=BATCH_SIZE)

  history = learner.fit(LR, n_cycles=EPOCHS, checkpoint_folder='/tmp')
  fold_accuracies = history.history['val_accuracy'] 
  best_score, best_epoch = max(fold_accuracies), np.array(fold_accuracies).argmax() + 1
  oof_loss_score.append(best_score)
  print(f'\033[1m\033[92m Fold {i+1}: {best_score}\33[0m\n')

  #Load best weights
  model = t.get_classifier()
  model.load_weights('../tmp/weights-0' + str(best_epoch) + '.hdf5')
  learner = ktrain.get_learner(model, train_data=train_set, val_data=val_set, batch_size=BATCH_SIZE)

  # Make predictions
  preds = ktrain.get_predictor(learner.model, preproc=t).predict(test_data, return_proba=True)

  # Append preds to oof_preds list
  oof_preds.append(preds)

# Check cv score and prepare submission file
LOSS = np.round(np.mean(oof_loss_score), 5)
print(f'\n\33[96m\33[1m\33[4m Mean Loss: {LOSS}\33[0m')

name = f'{MODEL_NAME}_ML{MAX_LEN}_BS{BATCH_SIZE}_FD{FOLDS}_LR{LR}_EP{EPOCHS}_LS{LOSS}'
sub = pd.DataFrame(np.mean(oof_preds, axis=0), columns = t.get_classes())
sub.to_csv(name + '.csv', index = False)
ss = pd.DataFrame({'ID':test.ID, 'label': sub.idxmax(axis = 1)})
ss.to_csv(f'KT_bert{LOSS}.csv', index = False)

Epoch 1/3
875/875 [==============================] - 292s 305ms/step - loss: 0.6345 - accuracy: 0.7121 - val_loss: 0.4663 - val_accuracy: 0.8023
Epoch 2/3
875/875 [==============================] - 273s 301ms/step - loss: 0.4152 - accuracy: 0.8300 - val_loss: 0.4754 - val_accuracy: 0.8016
Epoch 3/3
875/875 [==============================] - 274s 301ms/step - loss: 0.3116 - accuracy: 0.8760 - val_loss: 0.4404 - val_accuracy: 0.8218
 Fold 1: 0.8217856884002686

Epoch 1/3
875/875 [==============================] - 291s 305ms/step - loss: 0.6398 - accuracy: 0.7110 - val_loss: 0.4818 - val_accuracy: 0.8003
Epoch 2/3
875/875 [==============================] - 274s 302ms/step - loss: 0.4155 - accuracy: 0.8297 - val_loss: 0.4434 - val_accuracy: 0.8172
Epoch 3/3
875/875 [==============================] - 274s 302ms/step - loss: 0.3036 - accuracy: 0.8799 - val_loss: 0.4486 - val_accuracy: 0.8184
 Fold 2: 0.8184285759925842

Epoch 1/3
875/875 [==============================] - 292s 305ms/step - l